# Basics of network science (Network building blocks)

In [1]:
import networkx as nx
from matplotlib import pyplot as plt
import pandas as pd
%matplotlib inline

There are several python librariries that can be used to build and analyze networks. The most popular one is `networkx`. Other options are:
* [igraph](https://igraph.org/python/) - a library for creating and analyzing complex networks, written in C and Python.
* [graph-tool](https://graph-tool.skewed.de/) - a Python module for manipulation and statistical analysis of graphs (a.k.a. networks).
* [snap](http://snap.stanford.edu/snappy/index.html) - a general purpose, high performance system for analysis and manipulation of large networks.

You can also download [Gephis](https://gephi.org/) which is a visualization and exploration software for all kinds of graphs and networks. Gephi is open-source and free.

## Building blocks

Let's create an empty graph

In [ ]:
G = nx.Graph()

In [ ]:
G.add_edges_from([("A", "B"), ("A", "C"), ("B", "C")])
nx.draw(G, node_size=800, node_color="white", with_labels=True)

In [ ]:
G

In [ ]:
D = nx.DiGraph()
D.add_edges_from([("A", "B"), ("A", "C"), ("B", "C")])
nx.draw(D, node_size=800, node_color="white", with_labels=True)

OK, so these are pretty dull graphs. Networkx has a lot of built-in functions to generate more interesting graphs. For example, the `barabasi_albert_graph` function generates a scale-free graph based on the Barabási–Albert model.

In [ ]:
BA = nx.barabasi_albert_graph(100, 3)
nx.draw(BA, node_size=100)

You'll notice that the node positions are random. 

In [ ]:
nx.draw(BA, node_size=100, pos=nx.spring_layout(BA))

Positioning the nodes using a layout algorithm makes the graph more readable. Networkx has several layout algorithms, such as:
* `circular_layout` - positions nodes on a circle
* `random_layout` - positions nodes randomly
* `shell_layout` - positions nodes in concentric circles
* `spring_layout` - positions nodes using Fruchterman-Reingold force-directed algorithm
* `spectral_layout` - positions nodes using the eigenvectors of the graph Laplacian
* `planar_layout` - positions nodes without edge crossings
and more

In [ ]:
layouts = [
    nx.circular_layout,
    nx.random_layout,
    nx.shell_layout,
    nx.spring_layout,
    nx.spectral_layout,
]
for layout in layouts:
    fig, ax = plt.subplots()
    nx.draw(BA, node_size=100, pos=layout(BA), ax=ax)
    ax.set_title(layout.__name__)

Let's load a (sort-of) real-world network. The file `grays_anatomy.csv` contains a list of characters from the TV show Grey's Anatomy and the relationships between them. The file is in CSV format, with the first column containing the source node, the second column containing the target node, and the third column containing the weight of the edge.

In [ ]:
df_ga = pd.read_csv("./data/greys_anatomy.csv")
df_ga

In [ ]:
GA = nx.from_pandas_edgelist(df_ga, source="from", target="to")
nx.draw(GA, node_size=100, pos=nx.spring_layout(GA), with_labels=True)

## Node, edge and graph characteristics

### Node characteristics

When analyzing graphs, we have many ways to describe the nodes. The basic ones are:

* Degree - the number of edges connected to the node
* In-degree - the number of edges coming into the node (for directed graphs)
* Out-degree - the number of edges going out of the node (for directed graphs)

Later on, we will see more advanced node characteristics, such as centrality measures.

In [ ]:
degrees = pd.Series(dict(GA.degree())).sort_values(ascending=False)
degrees

Let's visualize the degree of each node in the Grey's Anatomy network

In [ ]:
nx.draw(
    GA,
    node_size=[GA.degree(node) * 50 for node in GA.nodes()],
    pos=nx.spring_layout(GA),
    with_labels=True,
)

From time to time, we may assign attributes to the nodes. For example, our data contains the gender of each character in the `from_gender` and `to_gender` columns. We can add this information to the graph.

In [ ]:
genders = dict()
for _, row in df_ga.iterrows():
    genders[row["from"]] = row["from_gender"]
    genders[row["to"]] = row["to_gender"]
# assign the gender to the nodes
nx.set_node_attributes(GA, genders, "gender")
gender_colors = {"M": "#ADD8E6", "F": "#FFB6C1"}  # light blue  # light pink
node_colors = [gender_colors[GA.nodes[node]["gender"]] for node in GA.nodes()]

In [ ]:
nx.draw(
    GA,
    node_size=[GA.degree(node) * 50 for node in GA.nodes()],
    node_color=node_colors,
    pos=nx.spring_layout(GA),
    with_labels=True,
)

## Edge attributes

We can also carry attributes on the edges. For example, we can add the weight of the edge to the graph.
Later on, we will see more advanced edge characteristics, such as edge betweenness, edge closeness, and edge clustering.

In [ ]:
GA = nx.from_pandas_edgelist(df_ga, source="from", target="to", edge_attr=["seriousness", 'is_cheat'])

In [ ]:
cheat_colors = {
    'yes': '#FFA07A',  # light salmon
    'no': '#98FB98'    # pale green
}
edge_colors = [cheat_colors[GA.edges[edge]["is_cheat"]] for edge in GA.edges()]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), dpi=100)
nx.draw(
    GA,
    node_size=[GA.degree(node) * 50 for node in GA.nodes()],
    node_color=node_colors,
    width=[GA.edges[edge]["seriousness"] * 0.5 for edge in GA.edges()],
    pos = nx.spring_layout(GA),
    edge_color=edge_colors,
    with_labels=True,
    ax=ax
)

## Graph characteristics

We saw that we can characterize the nodes and edges of a graph. We can also describe the graph as a whole. Some of the basic graph characteristics are:
* Number of nodes
* Number of edges
* Density - the ratio of the **actual** number of edges to the number of **theoretically** possible edges
* Diameter - the longest shortest path between any two nodes (we'll talk about shortest paths later)
* Degree distribution - the distribution of the degrees of the nodes in the graph

In [ ]:
def graph_summary(G):
    ret = dict()
    ret["Number of nodes"] = G.number_of_nodes()
    ret["Number of edges"] = G.number_of_edges()
    ret["Density"] = nx.density(G)
    ret["Diameter"] = nx.diameter(G) if nx.is_connected(G) else "Graph is not connected"
    return ret
print(graph_summary(GA))